<a href="https://colab.research.google.com/github/HaswanthVIT/Vitamin_Detection/blob/main/Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Testing With PDF**

In [ ]:
#  !pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git" -q
# !pip install --no-deps "trl<0.9.0" peft accelerate bitsandbytes xformers datasets -q
# !pip install pdfplumber -q

import unsloth
print('Unsloth installed successfully')

# 1. Mount Google Drive and load your model
from google.colab import drive
drive.mount('/content/drive')

# Set your model directory path (update if needed)
model_dir = "/content/drive/MyDrive/pdf2bits_final_model"

from transformers import AutoTokenizer
from unsloth import FastLanguageModel

tokenizer = AutoTokenizer.from_pretrained(model_dir)
model, _ = FastLanguageModel.from_pretrained(model_name=model_dir, max_seq_length=1024)

# 2. Upload PDF
from google.colab import files
uploaded = files.upload()
pdf_path = list(uploaded.keys())[0]

# 3. Extract text from PDF
import pdfplumber

def extract_text_from_pdf(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text + "\n"
    return text

pdf_text = extract_text_from_pdf(pdf_path)
print("Extracted PDF text length:", len(pdf_text))

# 4. Semantic (sentence-based) chunking for summarization
import re

def simple_sentence_tokenize(text):
    sentences = re.split(r'(?<=[.!?])\s+', text)
    return [s.strip() for s in sentences if s.strip()]

def semantic_chunk_text(text, max_words=150, min_words=50):
    sentences = simple_sentence_tokenize(text)
    chunks = []
    current_chunk = []
    current_length = 0

    for sentence in sentences:
        sentence_length = len(sentence.split())
        if current_length + sentence_length <= max_words:
            current_chunk.append(sentence)
            current_length += sentence_length
        else:
            if current_chunk and current_length >= min_words:
                chunks.append(' '.join(current_chunk))
                current_chunk = [sentence]
                current_length = sentence_length
            else:
                current_chunk.append(sentence)
                current_length += sentence_length

    if current_chunk:
        chunks.append(' '.join(current_chunk))

    return chunks

chunks = semantic_chunk_text(pdf_text, max_words=150, min_words=50)
print(f"Total semantic chunks for summarization: {len(chunks)}")

# 5. Prompt template and post-processing workaround
def make_prompt(chunk):
    return (
        "<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n"
        "Generate a 10-word educational summary for this text:\n\n"
        f"{chunk}\n"
        "<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n"
    )
import re

def extract_summary_from_output(output):
    marker = "Here is a 10-word educational summary for this text:"
    idx = output.find(marker)
    if idx != -1:
        # Get everything after the marker
        after_marker = output[idx + len(marker):].strip()
        # The summary is usually on the next non-empty line
        for line in after_marker.split('\n'):
            clean_line = line.strip().strip('"')
            if clean_line:
                # Optionally: limit to 10 words
                return ' '.join(clean_line.split()[:10])
    return ""

summaries = []
for chunk in chunks:
    prompt = make_prompt(chunk)
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=50, use_cache=True)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    summary = extract_summary_from_output(generated_text)
    if summary:
        summaries.append(summary)

for s in summaries:
    print(s)



  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Unsloth installed successfully
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
==((====))==  Unsloth 2025.6.12: Fast Llama patching. Transformers: 4.53.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Saving 1-Nature of Software Software Engineering Software process project product Process Models-14-12-2024-27-48.pdf to 1-Nature of Software Software Engineering Software process project product Process Models-14-12-2024-27-48.pdf
Extracted PDF text length: 15347
Total semantic chunks for summarization: 18
Software Engineering: Application of Scientific and Practical Knowledge to Software
Understand the problem before building a solution with modern complexity.
Software engineering tools automate software development processes and improve efficiency.
A software process framework sets the groundwork for software development.
This helps understand project goals and requirements through planning and
Developing a Food Delivery App Involves Planning and Testing Requirements
Software Project Management: Planning, Tracking, and Controlling Software Development Activities
Work Product Preparation and Production: Encompasses Work Product Management Processes
Process flow describes different ex